## Merging Dataframes
![image](https://shanelynnwebsite-mid9n9g1q9y8tt.netdna-ssl.com/wp-content/uploads/2017/03/pandas-merge-join-different-variable-names-copy-e1488722312527.png)

### Data

- [user_usage.csv](https://github.com/shanealynn/Pandas-Merge-Tutorial/blob/master/user_usage.csv) - A first dataset containing users monthly mobile usage statistics
- [user_device.csv](https://github.com/shanealynn/Pandas-Merge-Tutorial/blob/master/user_device.csv) - A second dataset containing details of an individual “use” of the system, with dates and device information.
- [android_devices.csv](https://github.com/shanealynn/Pandas-Merge-Tutorial/blob/master/android_devices.csv) - A third dataset with device and manufacturer data, which lists all Android devices and their model code, obtained from Google 

We can load these CSV files as Pandas DataFrames into pandas using the Pandas `read_csv` command, and examine the contents using the DataFrame `head()` command.

In [1]:
import pandas as pd

In [2]:
user_usage = pd.read_csv('https://raw.githubusercontent.com/shanealynn/Pandas-Merge-Tutorial/master/user_usage.csv')
user_usage.head()

outgoing_mins_per_month  outgoing_sms_per_month  monthly_mb  use_id
0                    21.97                    4.82     1557.33   22787
1                  1710.08                  136.88     7267.55   22788
2                  1710.08                  136.88     7267.55   22789
3                    94.46                   35.17      519.12   22790
4                    71.59                   79.26     1557.33   22792

In [3]:
user_device = pd.read_csv('https://raw.githubusercontent.com/shanealynn/Pandas-Merge-Tutorial/master/user_device.csv')
user_device.head()

use_id  user_id platform  platform_version     device  use_type_id
0   22782    26980      ios              10.2  iPhone7,2            2
1   22783    29628  android               6.0    Nexus 5            3
2   22784    28473  android               5.1   SM-G903F            1
3   22785    15200      ios              10.2  iPhone7,2            3
4   22786    28239  android               6.0  ONE E1003            1

In [4]:
devices = pd.read_csv('https://raw.githubusercontent.com/shanealynn/Pandas-Merge-Tutorial/master/android_devices.csv')
devices.head()

Retail Branding Marketing Name    Device                      Model
0             NaN            NaN    AD681H  Smartfren Andromax AD681H
1             NaN            NaN     FJL21                      FJL21
2             NaN            NaN       T31              Panasonic T31
3             NaN            NaN  hws7721g         MediaPad 7 Youth 2
4              3Q        OC1020A   OC1020A                    OC1020A

### First merge

We're trying to get the average usage figures for different types of devices. So we need to get the user's device code from `user_usage` as a column on `user_usage`, and then get the device's manufacturer from `devices` as a column on the result.

First, we merge `user_usage` with `user_device` with `use_id` as our common column

In [5]:
result = pd.merge(user_usage,
                 user_device[['use_id', 'platform', 'device']],
                 on='use_id')
result.head()

outgoing_mins_per_month  outgoing_sms_per_month  monthly_mb  use_id  \
0                    21.97                    4.82     1557.33   22787   
1                  1710.08                  136.88     7267.55   22788   
2                  1710.08                  136.88     7267.55   22789   
3                    94.46                   35.17      519.12   22790   
4                    71.59                   79.26     1557.33   22792   

  platform    device  
0  android  GT-I9505  
1  android  SM-G930F  
2  android  SM-G930F  
3  android     D2303  
4  android  SM-G361F

While on first glance, this may seem perfect, all is not as it seems.

In [6]:
print("user_usage dimensions: {}".format(user_usage.shape))
print("user_device dimensions: {}".format(user_device[['use_id', 'platform', 'device']].shape))
print("result dimensions: {}".format(result.shape))

user_usage dimensions: (240, 4)
user_device dimensions: (272, 3)
result dimensions: (159, 6)


In [7]:
user_usage['use_id'].isin(user_device['use_id']).value_counts()

True     159
False     81
Name: use_id, dtype: int64

### Left merge example

A left merge, or left join, between two dataframes keeps all of the rows and values from the left dataframe, in this case `user_usage`.

Rows from the right dataframe will be kept in the result only where there is a match in the merge variable in the right dataframe, and `NaN` values will be in the result where not.

In [8]:
result = pd.merge(user_usage,
                 user_device[['use_id', 'platform', 'device']],
                 on='use_id', how='left')
print("user_usage dimensions: {}".format(user_usage.shape))
print("result dimensions: {}".format(result.shape))
print("There are {} missing values in the result.".format(
        result['device'].isnull().sum()))

user_usage dimensions: (240, 4)
result dimensions: (240, 6)
There are 81 missing values in the result.


In [9]:
result.head()

outgoing_mins_per_month  outgoing_sms_per_month  monthly_mb  use_id  \
0                    21.97                    4.82     1557.33   22787   
1                  1710.08                  136.88     7267.55   22788   
2                  1710.08                  136.88     7267.55   22789   
3                    94.46                   35.17      519.12   22790   
4                    71.59                   79.26     1557.33   22792   

  platform    device  
0  android  GT-I9505  
1  android  SM-G930F  
2  android  SM-G930F  
3  android     D2303  
4  android  SM-G361F

In [10]:
result.tail()

outgoing_mins_per_month  outgoing_sms_per_month  monthly_mb  use_id  \
235                   260.66                   68.44      896.96   25008   
236                    97.12                   36.50     2815.00   25040   
237                   355.93                   12.37     6828.09   25046   
238                   632.06                  120.46     1453.16   25058   
239                   488.70                  906.92     3089.85   25220   

    platform device  
235      NaN    NaN  
236      NaN    NaN  
237      NaN    NaN  
238      NaN    NaN  
239      NaN    NaN

### Right merge example

A right merge, or right join, between two dataframes keeps all of the rows and values from the right dataframe, in this case `user_device`.

Rows from the left dataframe will be kept where there is a match in the merge variable, and `NaN` values will be in the result where not.

In [11]:
result = pd.merge(user_usage,
                 user_device[['use_id', 'platform', 'device']],
                 on='use_id', how='right')
print("user_device dimensions: {}".format(user_device.shape))
print("result dimensions: {}".format(result.shape))
print("There are {} missing values in the 'monthly_mb' column in the result.".format(
        result['monthly_mb'].isnull().sum()))
print("There are {} missing values in the 'platform' column in the result.".format(
        result['platform'].isnull().sum()))

user_device dimensions: (272, 6)
result dimensions: (272, 6)
There are 113 missing values in the 'monthly_mb' column in the result.
There are 0 missing values in the 'platform' column in the result.


### Outer merge example

A full outer join, or outer merge, keeps all rows from the left and right dataframe in the result. Rows will be aligned where there is shared join values between the left and right, and rows with `NaN` values, in either the left-originating or right-originating columns will be, will be left in the result where there is no shared join value.

In the final result, a subset of rows should have no missing values. These rows are the rows where there was a match between the merge column in the left and right dataframes. These rows are the same values as found by our inner merge result before.

In [12]:
print("There are {} unique values of use_id in our dataframes.".format(
        pd.concat([user_usage['use_id'], user_device['use_id']]).unique().shape[0]))
result = pd.merge(user_usage,
                 user_device[['use_id', 'platform', 'device']],
                 on='use_id', how='outer', indicator=True)

print("Outer merge result has {} rows.".format(result.shape))

print("There are {} rows with no missing values.".format(
    (result.apply(lambda x: x.isnull().sum(), axis=1) == 0).sum()))

There are 353 unique values of use_id in our dataframes.
Outer merge result has (353, 7) rows.
There are 159 rows with no missing values.


In [13]:
result.iloc[[0, 1, 200,201, 350,351]]

outgoing_mins_per_month  outgoing_sms_per_month  monthly_mb  use_id  \
0                      21.97                    4.82     1557.33   22787   
1                    1710.08                  136.88     7267.55   22788   
200                    28.79                   29.42     3114.67   23988   
201                   616.56                   99.85     5414.14   24006   
350                      NaN                     NaN         NaN   23050   
351                      NaN                     NaN         NaN   23051   

    platform     device      _merge  
0    android   GT-I9505        both  
1    android   SM-G930F        both  
200      NaN        NaN   left_only  
201      NaN        NaN   left_only  
350      ios  iPhone7,2  right_only  
351      ios  iPhone7,2  right_only

### Final merge - adding device manufacturer

In [14]:
# First, add the platform and device to the user usage.
result = pd.merge(user_usage,
                 user_device[['use_id', 'platform', 'device']],
                 on='use_id',
                 how='left')

# Now, based on the "device" column in result, match the "Model" column in devices.
devices.rename(columns={"Retail Branding": "manufacturer"}, inplace=True)
result = pd.merge(result, 
                  devices[['manufacturer', 'Model']],
                  left_on='device',
                  right_on='Model',
                  how='left')

result.head()

outgoing_mins_per_month  outgoing_sms_per_month  monthly_mb  use_id  \
0                    21.97                    4.82     1557.33   22787   
1                  1710.08                  136.88     7267.55   22788   
2                  1710.08                  136.88     7267.55   22789   
3                    94.46                   35.17      519.12   22790   
4                    71.59                   79.26     1557.33   22792   

  platform    device manufacturer     Model  
0  android  GT-I9505      Samsung  GT-I9505  
1  android  SM-G930F      Samsung  SM-G930F  
2  android  SM-G930F      Samsung  SM-G930F  
3  android     D2303         Sony     D2303  
4  android  SM-G361F      Samsung  SM-G361F

In [15]:
devices[devices.Model == 'SM-G930F']

manufacturer Marketing Name   Device     Model
10381      Samsung      Galaxy S7  herolte  SM-G930F

### Calculating statistics on final result
With merges complete, we can simply calculate statistics for users grouped by the manufacturer of their device.

In [16]:
result.head()

outgoing_mins_per_month  outgoing_sms_per_month  monthly_mb  use_id  \
0                    21.97                    4.82     1557.33   22787   
1                  1710.08                  136.88     7267.55   22788   
2                  1710.08                  136.88     7267.55   22789   
3                    94.46                   35.17      519.12   22790   
4                    71.59                   79.26     1557.33   22792   

  platform    device manufacturer     Model  
0  android  GT-I9505      Samsung  GT-I9505  
1  android  SM-G930F      Samsung  SM-G930F  
2  android  SM-G930F      Samsung  SM-G930F  
3  android     D2303         Sony     D2303  
4  android  SM-G361F      Samsung  SM-G361F

In [17]:
result.groupby("manufacturer").agg({
        "outgoing_mins_per_month": "mean",
        "outgoing_sms_per_month": "mean",
        "monthly_mb": "mean",
        "use_id": "count"
    })

outgoing_mins_per_month  outgoing_sms_per_month    monthly_mb  \
manufacturer                                                                  
HTC                        299.842955               93.059318   5144.077955   
Huawei                      81.526667                9.500000   1561.226667   
LGE                        111.530000               12.760000   1557.330000   
Lava                        60.650000              261.900000  12458.670000   
Lenovo                     215.920000               12.930000   1557.330000   
Motorola                    95.127500               65.666250   3946.500000   
OnePlus                    354.855000               48.330000   6575.410000   
Samsung                    191.010093               92.390463   4017.318889   
Sony                       177.315625               40.176250   3212.000625   
Vodafone                    42.750000               46.830000   5191.120000   
ZTE                         42.750000               46.830000   5191.120000   

              use_id  
manufacturer          
HTC               44  
Huawei             3  
LGE                2  
Lava               2  
Lenovo             2  
Motorola          16  
OnePlus            6  
Samsung          108  
Sony              16  
Vodafone           1  
ZTE                1